In [1]:
%load_ext autoreload
%autoreload 2

In [67]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import tensorflow
import pickle
import os
from  fobokiller.cosim import load_embedding, model, compute_sim_df
from joblib import Parallel,delayed
import multiprocessing

In [68]:
embedding = load_embedding()

In [69]:
%%time
compute_sim_df("I want a burger", embedding)

CPU times: user 1.6 s, sys: 4.74 s, total: 6.33 s
Wall time: 1min 16s


,alias,date,rate,review_clean,review_sentences,sim_s,rate_filtered,review_filtered,sim_r,rate_all,review_all,ratio,metric
0,breakfast-in-america-2-paris,2017-05-04,4,the reason yelp exists. what a wonderful disco...,want a proper hamburger,0.738675,4.157895,16,0.480181,3.881956,82,0.195122,0.077914
1,pny-burger-oberkampf-paris,2019-06-05,2,i love burgers. my go to meal: burger. this pl...,i love burgers,0.713686,3.444444,12,0.511917,3.844444,18,0.666667,0.235103
2,l-arpège-paris-2,2016-01-19,3,l'arpege is france's -michelin starred restaur...,i want beef,0.705174,3.285714,6,0.521521,3.808692,152,0.039474,0.013528
3,hippopotamus-paris-13,2018-05-28,5,in paris with my husband for my honeymoon. we ...,get the burger,0.703004,3.750000,4,0.504269,2.227273,21,0.190476,0.072038
4,h-a-n-d-paris-2,2017-07-04,1,absolutly horrid. ordered a burger only to bit...,if you want a burger you're better off going ...,0.698962,2.545455,7,0.468972,2.751678,20,0.350000,0.083562
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,le-relais-de-venise-l-entrecôte-paris-2,2015-02-06,4,"steak, fries, wine. questions? without a doubt...","steak, fries, wine",0.407238,4.304348,23,0.442410,4.208922,142,0.161972,0.061688
2996,le-chateaubriand-paris,2016-06-18,3,"dinner for . - ambiance () cozy and modern, bu...",dinner for,0.407228,4.333333,3,0.409406,3.808278,163,0.018405,0.006530
2997,le-minipalais-paris,2011-08-05,3,dinner for . the apps were delicious and flavo...,dinner for,0.407228,2.000000,3,0.417131,3.399132,43,0.069767,0.011641
2998,les-galopins-paris,2018-09-20,5,called the restaurant and between our language...,"more like dinner for """"",0.407223,5.000000,1,0.466899,4.816327,5,0.200000,0.093380


In [42]:
input_encoded = model.encode(text)
embedding_split = np.array_split(embedding,
                                     multiprocessing.cpu_count() - 1)

similarities_parra = Parallel(n_jobs=-1)(
        delayed(util.cos_sim)(input_encoded, split_embedding)
        for split_embedding in embedding_split)
    # // powa !
similarities = tensorflow.concat(similarities_parra, 1)
    #similarities = util.cos_sim(input_encoded, np.array(embedding))

In [44]:
similarities_old = util.cos_sim(input_encoded, np.array(embedding))

In [62]:
import torch

In [65]:
torch.cat(similarities_parra,1)

tensor([[0.1825, 0.1664, 0.2353,  ..., 0.3719, 0.2296, 0.0928]])

In [61]:
similarities_old

tensor([[0.1825, 0.1664, 0.2353,  ..., 0.3719, 0.2296, 0.0928]])

In [ ]:
df_sim = df_full.assign(sim=similarities.T)

In [39]:
def compute_sim_df(text, embedding, n_prox=3000, min_review=0):
    input_encoded = model.encode(text)
    embedding_split = np.array_split(embedding,
                                     multiprocessing.cpu_count() - 1)

    similarities_parra = Parallel(n_jobs=-1)(
        delayed(util.cos_sim)(input_encoded, split_embedding)
        for split_embedding in embedding_split)
    # // powa !
    similarities = tensorflow.concat(similarities_parra, 1)
    #similarities = util.cos_sim(input_encoded, np.array(embedding))

    df_sim = df_full.assign(sim=similarities.T)

    if n_prox:
        df_sentences = df_sim.sort_values('sim', ascending=False)[:n_prox]
    else:
        df_sentences = df_sim.sort_values('sim', ascending=False)

    df_agg = df_sentences.groupby('alias').agg({
        'rate': 'mean',
        'review': 'nunique',
        #'review_sentences':'first',
        #'review_clean':lambda txt: ' // '.join(txt),
        'sim': 'mean'
    })

    df_resto = df_agg.merge(df_all_resto,
                            on='alias',
                            how='left',
                            suffixes=('_filtered', '_all')).reset_index()

    df_resto['ratio'] = df_resto['review_filtered'] / df_resto['review_all']

    df_resto = df_resto.sort_values('ratio')
    df_resto = df_resto[df_resto['review_all'] > min_review]

    df_final = df_sentences.merge(df_resto,
                                  on='alias',
                                  how='left',
                                  suffixes=('_s', '_r'))

    df_final['metric'] = df_final['ratio'] * df_final['sim_r'] * df_final[
        'rate_filtered'] / 5

    df_final.drop(columns=['review'], inplace=True)

    return df_final

In [40]:
%%time
A =compute_sim_df("I want a burger",embedding)

AttributeError: 
        'EagerTensor' object has no attribute 'T'.
        If you are looking for numpy-related methods, please run the following:
        from tensorflow.python.ops.numpy_ops import np_config
        np_config.enable_numpy_behavior()

In [5]:
%%time
A =compute_sim_df("I want a burger",embedding)

In [6]:
embedding = load_embedding()

2021-12-02 14:44:55.884637: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
df_all_resto = df_full.groupby('alias').agg({ 'rate':'mean',
                                                'review':'nunique'
                                            })

In [8]:
input_encoded = model.encode("I want a burger")

In [70]:
%%time
os.environ["TOKENIZERS_PARALLELISM"] = "True"
similarities_simple = util.cos_sim(input_encoded, np.array(embedding))

CPU times: user 5.5 s, sys: 19.5 s, total: 25 s
Wall time: 2min


In [71]:
%%time
os.environ["TOKENIZERS_PARALLELISM"] = "false"
similarities_simple = util.cos_sim(input_encoded, np.array(embedding))

CPU times: user 6.28 s, sys: 20.9 s, total: 27.1 s
Wall time: 1min 4s


In [14]:
### IMPORTS ###

#canonical
from sys import path
import pandas as pd
import numpy as np
import os

#words/sentences  preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf


#modeling
from tensorflow.keras import models

#cosim

from fobokiller.cosim import load_embedding,compute_sim_df,summary_reviews

def load_model():
    #path_model = os.path.join(os.path.dirname(os.path.dirname(__file__)),'model_heatmap')
    model = models.load_model("model_heatmap/")
    return model

def load_reviews_dataset():
    path_reviews_all = os.path.join(os.path.dirname(__file__),
                                    'data/scrapping_cleaned_sentences.csv')
    dataset = pd.read_csv(path_reviews_all,index_col=0)
    embedding=load_embedding()
    dataset = dataset.assign(embedding=[*np.array(embedding)])
    dataset = dataset.groupby('review_clean').agg({
                                                'alias': 'first',
                                                'rate': 'mean',
                                                'review_sentences': list,
                                                'embedding': list,
                                            }).reset_index()
    dataset['review_sentences_trimed'] = dataset['review_sentences'].apply(
        lambda list_: list_[:30])

    return dataset


def creating_query_dataset(embedding,reviews_dataset,query, n_prox, min_review,n_best):
    #loading data
    results = compute_sim_df(query,embedding,n_prox=n_prox,min_review=min_review)
    summary = summary_reviews(results,n_best=n_best)

    #creating is_sim & is_in_summary columns
    results['is_sim'] = 1
    summary['is_in_summary'] = 1

    #merging datasets
    results_trimed = results.drop(columns=['alias', 'rate', 'review_sentences'])
    tmp_df = reviews_dataset.merge(results_trimed, on='review_clean', how='left')
    tmp_df['is_sim'].fillna(0, inplace=True)
    all_df = tmp_df.merge(summary, on='alias', how='left')
    all_df.fillna(0, inplace=True)

    return all_df



def heatmap_sentences(review_sentences, review_embedded, model):
    #padding
    review_embedded = pad_sequences([review_embedded],
                                    dtype='float32',
                                    padding='post',
                                    maxlen=30)
    # predict
    preds = model.predict(review_embedded)
    #gradient tape
    with tf.GradientTape() as tape:
        class_idx = np.argmax(preds[0]) #a priori useless always return 0
        last_conv_layer = model.get_layer('conv1d')
        iterate = tf.keras.models.Model([model.inputs],
                                        [model.output, last_conv_layer.output])
        model_out, last_conv_layer = iterate(review_embedded)
        class_out = model_out[:, class_idx]
        grads = tape.gradient(class_out, last_conv_layer)
        pooled_grads = tf.reduce_mean(grads)

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer),
                             axis=-1)
    heatmap = np.maximum(heatmap, 0)
    if np.max(heatmap) !=0: ### A VOIR LE SENS ?!
        heatmap /= np.max(heatmap)

    polarity_distance=np.max(heatmap[0])-np.min(heatmap[0])

    html = ""
    for i, j in enumerate(review_sentences):
        html += f"<span style='background-color:rgba(0,{255-heatmap[0][i]*255},0,0.6)'>{j} </span>"

    return html,polarity_distance

model_heatmap = load_model()

def apply_heatmap_html(df):
    html_out,polarity_distance=heatmap_sentences(df['review_sentences_trimed'],
                                                 df['embedding'],model_heatmap)
    return html_out


def apply_heatmap_polarity(df):
    html_out, polarity_distance = heatmap_sentences(
        df['review_sentences_trimed'], df['embedding'], model_heatmap)
    return polarity_distance





In [16]:
def sr2(text, n_best=10, n_prox=3000, min_review=10):

    #setting types
    min_review = int(min_review)
    if pd.isna(n_prox):
        pass
    else:
        n_prox = int(n_prox)
    n_best = int(n_best)

    #loading datasets
    results = compute_sim_df(text, embedding, n_prox, min_review)
    summary = summary_reviews(results, n_best)
    results['is_sim'] = 1
    summary['is_in_summary'] = 1
    results_trimed = results.drop(
        columns=['alias', 'rate', 'review_sentences'])

    #merging datasets and house cleaning

    tmp_df = reviews_dataset.merge(results_trimed,
                                   on='review_clean',
                                   how='left')
    tmp_df['is_sim'].fillna(0, inplace=True)
    all_df = tmp_df.merge(summary, on='alias', how='left')
    all_df.fillna(0, inplace=True)
    all_df = all_df[all_df["is_in_summary"]==1]

    #apply heatmap for html and polarity score
    all_df['reviews_heatmaps_html'] = all_df.apply(apply_heatmap_html,axis=1)
    all_df['reviews_heatmaps_polarity'] = all_df.apply(apply_heatmap_polarity, axis=1)
    ####  metrics for the val of the request
    all_df['request_metric'] = all_df[(all_df['is_in_summary'] == 1) & (
        all_df['is_sim'] == 1)]['nb_sentences'].sum() *100 /3000
    summary_reconstructed = all_df.groupby('alias').agg({
        'review_clean':
        list,
        'nb_sentences':
        'mean',
        'nb_review':
        'mean',
        'metric sim_ratio':
        'mean',
        'reviews_heatmaps_html':
        list,
        'reviews_heatmaps_polarity':
        list
    })

    #print("check")
    #print(type(summary_reconstructed))
    #print(summary_reconstructed.columns)
    #print(summary_reconstructed.shape)
    output_json = summary_reconstructed.to_dict()
    return output_json

In [18]:
%%time
sr2("I want a burger")

AttributeError: 
        'EagerTensor' object has no attribute 'T'.
        If you are looking for numpy-related methods, please run the following:
        from tensorflow.python.ops.numpy_ops import np_config
        np_config.enable_numpy_behavior()